In [1]:
import os
import numpy as np
import torch

from utils.util import get_mask_mnr, get_mask_bm, get_mask_rm
from utils.util import find_max_epoch, print_size, sampling, calc_diffusion_hyperparams

from imputers.SSSDS4Imputer import SSSDS4Imputer

from sklearn.metrics import mean_squared_error
from statistics import mean

CUDA extension for cauchy multiplication not found. Install by going to extensions/cauchy/ and running `python setup.py install`. This should speed up end-to-end training by 10-50%
Falling back on slow Cauchy kernel. Install at least one of pykeops or the CUDA extension for efficiency.


In [2]:
global trainset_config
global diffusion_hyperparams
global model_config

In [3]:
config = {   
    "diffusion_config":{
        "T": 200,
        "beta_0": 0.0001,
        "beta_T": 0.02
    },
    "wavenet_config": {
        "in_channels": 14, 
        "out_channels":14,
        "num_res_layers": 36,
        "res_channels": 256, 
        "skip_channels": 256,
        "diffusion_step_embed_dim_in": 128,
        "diffusion_step_embed_dim_mid": 512,
        "diffusion_step_embed_dim_out": 512,
        "s4_lmax": 100,
        "s4_d_state":64,
        "s4_dropout":0.0,
        "s4_bidirectional":1,
        "s4_layernorm":1
    },
    "train_config": {
        "output_directory": "./results/mujoco",
        "ckpt_iter": -1,
        "iters_per_ckpt": 100,
        "iters_per_logging": 100,
        "n_iters": 150000,
        "learning_rate": 2e-4,
        "only_generate_missing": 1,
        "use_model": 2,
        "masking": "rm",
        "missing_k": 90
    },
    "trainset_config":{
        "train_data_path": "/home/hanyuji/data/mujoco_dataset/train_mujoco.npy",
        "test_data_path": "/home/hanyuji/data/mujoco_dataset/test_mujoco.npy",
        "segment_length":100,
        "sampling_rate": 100
    },
    "gen_config":{
        "output_directory": "./results/mujoco_output",
        "ckpt_path": "./results/mujoco/"
    }
}




gen_config = config['gen_config']
train_config = config["train_config"]  # training parameters
trainset_config = config["trainset_config"]  # to load trainset
diffusion_hyperparams = calc_diffusion_hyperparams(**config["diffusion_config"])  # dictionary of all diffusion hyperparameters
model_config = config['wavenet_config']

In [4]:
output_directory = "./results/mujoco_output"
ckpt_path = "./results/mujoco/"
num_samples = 500
data_path = trainset_config["test_data_path"]
ckpt_iter = 'max'
masking = train_config["masking"]
missing_k = train_config["missing_k"]
only_generate_missing = train_config["only_generate_missing"]
    
"""
Generate data based on ground truth 

Parameters:
output_directory (str):           save generated speeches to this path
num_samples (int):                number of samples to generate, default is 4
ckpt_path (str):                  checkpoint path
ckpt_iter (int or 'max'):         the pretrained checkpoint to be loaded; 
                                    automitically selects the maximum iteration if 'max' is selected
data_path (str):                  path to dataset, numpy array.
use_model (int):                  0:DiffWave. 1:SSSDSA. 2:SSSDS4.
masking (str):                    'mnr': missing not at random, 'bm': black-out, 'rm': random missing
only_generate_missing (int):      0:all sample diffusion.  1:only apply diffusion to missing portions of the signal
missing_k (int)                   k missing time points for each channel across the length.
"""


"\nGenerate data based on ground truth \n\nParameters:\noutput_directory (str):           save generated speeches to this path\nnum_samples (int):                number of samples to generate, default is 4\nckpt_path (str):                  checkpoint path\nckpt_iter (int or 'max'):         the pretrained checkpoint to be loaded; \n                                    automitically selects the maximum iteration if 'max' is selected\ndata_path (str):                  path to dataset, numpy array.\nuse_model (int):                  0:DiffWave. 1:SSSDSA. 2:SSSDS4.\nmasking (str):                    'mnr': missing not at random, 'bm': black-out, 'rm': random missing\nonly_generate_missing (int):      0:all sample diffusion.  1:only apply diffusion to missing portions of the signal\nmissing_k (int)                   k missing time points for each channel across the length.\n"

In [5]:
# map diffusion hyperparameters to gpu
for key in diffusion_hyperparams:
    if key != "T":
        diffusion_hyperparams[key] = diffusion_hyperparams[key].cuda()

        
# predefine model
net = SSSDS4Imputer(**model_config).cuda()
print_size(net)


# load checkpoint
if ckpt_iter == 'max':
    ckpt_iter = find_max_epoch(ckpt_path)
model_path = os.path.join(ckpt_path, f'{ckpt_iter}.pkl')
try:
    checkpoint = torch.load(model_path, map_location='cpu')
    net.load_state_dict(checkpoint['model_state_dict'])
    print(f'Successfully loaded model at iteration {ckpt_iter}')
except:
    raise Exception('No valid model found')


SSSDS4Imputer Parameters: 48.371726M


FileNotFoundError: [Errno 2] No such file or directory: './results/mujoco/'

In [6]:
### Custom data loading and reshaping ###

testing_data = np.load(trainset_config['test_data_path'])
testing_data = np.split(testing_data, 4, 0)
testing_data = np.array(testing_data)
testing_data = torch.from_numpy(testing_data).float().cuda()
print('Data loaded')


Data loaded


In [7]:
all_mse = []


for i, batch in enumerate(testing_data):

    if masking == 'mnr':
        mask_T = get_mask_mnr(batch[0], missing_k)
        mask = mask_T.permute(1, 0)
        mask = mask.repeat(batch.size()[0], 1, 1)
        mask = mask.type(torch.float).cuda()

    elif masking == 'bm':
        mask_T = get_mask_bm(batch[0], missing_k)
        mask = mask_T.permute(1, 0)
        mask = mask.repeat(batch.size()[0], 1, 1)
        mask = mask.type(torch.float).cuda()

    elif masking == 'rm':
        mask_T = get_mask_rm(batch[0], missing_k)
        mask = mask_T.permute(1, 0)
        mask = mask.repeat(batch.size()[0], 1, 1).float().cuda()

        
        
    batch = batch.permute(0,2,1)
    
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()

    sample_length = batch.size(2)
    sample_channels = batch.size(1)
    generated_audio = sampling(net, (num_samples, sample_channels, sample_length),
                                diffusion_hyperparams,
                                cond=batch,
                                mask=mask,
                                only_generate_missing=only_generate_missing)

    end.record()
    torch.cuda.synchronize()

    print('generated {} utterances of random_digit at iteration {} in {} seconds'.format(num_samples,
                                                                                            ckpt_iter,
                                                                                            int(start.elapsed_time(
                                                                                                end) / 1000)))

    
    generated_audio = generated_audio.detach().cpu().numpy()
    batch = batch.detach().cpu().numpy()
    mask = mask.detach().cpu().numpy() 
    
    
    outfile = f'imputation{i}.npy'
    new_out = os.path.join(output_directory, outfile)
    np.save(new_out, generated_audio)

    outfile = f'original{i}.npy'  
    new_out = os.path.join(output_directory, outfile)
    np.save(new_out, batch)

    outfile = f'mask{i}.npy'
    new_out = os.path.join(output_directory, outfile)
    np.save(new_out, mask)

    print('saved generated samples at iteration %s' % ckpt_iter)
    
    mse = mean_squared_error(generated_audio[~mask.astype(bool)], batch[~mask.astype(bool)])
    all_mse.append(mse)

print('Total MSE:', mean(all_mse))




begin sampling, total number of reverse steps = 200
generated 500 utterances of random_digit at iteration 23500 in 273 seconds
saved generated samples at iteration 23500
begin sampling, total number of reverse steps = 200
generated 500 utterances of random_digit at iteration 23500 in 270 seconds
saved generated samples at iteration 23500
begin sampling, total number of reverse steps = 200
generated 500 utterances of random_digit at iteration 23500 in 261 seconds
saved generated samples at iteration 23500
begin sampling, total number of reverse steps = 200
generated 500 utterances of random_digit at iteration 23500 in 261 seconds
saved generated samples at iteration 23500
Total MSE: 0.0046025217
